**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024

# Uranium Mill Network Dynamics Simulation Report
$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

**Student(s):** Alyssa Puglisi <br>
**Course:** Engy-4390 Nuclear Systems Design and Analysis  <br>
**Mentor:** Prof. Valmor F. de Almeida

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**16Oct2024**

[Project Repository on `GitHub`](https://github.com/dpploy/engy-4390/tree/main/projects/2024/u-mill)

---
## Table of Contents<a id="toc"></a>
* [Problem Statment](#prob)
* [Introduction](#intro)
* [Objectives](#obj)
* [Tools](#tools)
* [Approach and Dynamics Equations](#app)
* [Challenges & Course Corrections](#correct)
* [Initialization of Solvent Extraction Module](#IPM)
* [Startup](#su)
* [Steady-State](#ss)
* [Disturbance](#malfunction)
* [References](#refs)
---

## [Problem Statement](#toc)<a id="prob"></a>

## [Introduction](#toc)<a id="intro"></a>


## [Objectives](#toc)<a id="obj"></a>

## [Tools](#toc)<a id="tools"></a>

## [Approach and Dynamics Equations](#toc)<a id="app"></a>

## [Challenges & Course Correctons](#toc)<a id="correct"></a>

## [Initialization of Solvent Extraction Modules](#toc)<a id="IPM"></a>

In [12]:
```python
                                              |
                                              |  Ammonium Diuranate Feed
                                              v
                                       ________________       
                                      |                |
                                      |                |
                                      |                |<------ Steam Sparging
                                      |  Evaporation   |
                                      |                |
                   Off-Gas <----------|                |<------ Resistance Heating (Internal)
                                      |                |
                                      |----------------|
                      Off-Gas <-------|                |<------ Sweeping Gas
                                      |                |
                                      |   Calcination  |
                 Sweeping Gas <-------|                |
                   (Disposal)         |                |
                                      |________________|<------ Resistance Heating (Internal)
                                               |
                                               |
                                               |
                                               v
                                        Product (U3O8)
```

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)